In [3]:
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import descartes
import numpy as np
import seaborn as sns
import re

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
data = pd.read_csv("properatti.csv", sep = ",")

In [ ]:
data_1=data.drop(columns=['image_thumbnail','properati_url','lat-lon','Unnamed: 0','expenses','price_aprox_local_currency'])

In [ ]:
(data_1.isnull().sum()/data_1.shape[0])*100

In [ ]:
data_1.head(3)

In [ ]:
# number of nulls for each column
vc_nulls = data_1.apply(lambda x: x.isnull().value_counts()).T[True]
vc_nulls.hist() # if you want a histogram of these counts
# or if you wanted to plot the null count of each column as a bar
vc_nulls.plot(kind = 'bar')

### Filtramos monedas

In [ ]:
monedas=data_1['currency'].unique()
monedas

In [ ]:
data_1['currency'].value_counts()

In [ ]:
mask1=data['currency']=='UYU'
data_1[mask1]

In [ ]:
mask2=data['currency']=='PEN'
data_1[mask2]

In [ ]:
data_2=data_1.drop(index=[107390,50387,50388])

data_2['currency'].unique()

In [ ]:
data_2.head(2)

Entonces hay que usar la columna 'price_aprox_usd'

In [ ]:
maskarg=data_2['currency']=='ARS'

In [ ]:
data_2[maskarg]['price_aprox_usd'].isnull().sum()

### Borremos los datos duplicados para eso saquemos los que tengan igual lon, lat y descripción.

In [ ]:
data_2_nodup = data_2.drop_duplicates(subset = ['description',"lat", "lon"], keep = "first")

In [ ]:
data_2_nodup.info()

In [ ]:
data_2_nodup.head(5)

### Analicemos outliers de la columna 'price_aprox_usd'

In [ ]:
describe_price_aprox_usd = data_2_nodup["price_aprox_usd"].describe()
describe_price_aprox_usd

In [ ]:
q1 = describe_price_aprox_usd.quantile(0.25)
print(q1)
q2 = describe_price_aprox_usd.quantile(0.5)
print(q2)
q3 = describe_price_aprox_usd.quantile(0.75)
print(q3)


iqr = (q3 - q1) * 1.5

up_threshold = q3 + iqr
low_threshold = q1 - iqr

print(up_threshold)
print(low_threshold)

outlier_mask_up = describe_price_aprox_usd > up_threshold
outlier_mask_down = describe_price_aprox_usd < low_threshold
outlier_mask = np.logical_or(outlier_mask_up, outlier_mask_down)
not_outliers = np.logical_not(outlier_mask)

outliers = describe_price_aprox_usd[outlier_mask]

In [ ]:
describe_price_aprox_usd_filtered = describe_price_aprox_usd[not_outliers]
describe_price_aprox_usd_filtered.describe()

In [ ]:
describe_price_aprox_usd.describe()

In [ ]:
sns.boxplot(x=describe_price_aprox_usd.values);

In [ ]:
sns.boxplot(x=describe_price_aprox_usd_filtered.values);

# Imputar price usd per m2

In [ ]:
data_2_nodup['price_usd_per_m2_clean'] = data_2_nodup['price_usd_per_m2']
mask1 = data_2_nodup['price_usd_per_m2_clean'].isnull()

(data_2_nodup.loc[mask1, 'price_usd_per_m2_clean']).isnull().sum()

data_2_nodup.loc[mask1, 'price_usd_per_m2_clean'] = (data_2_nodup.loc[mask1, 'price']/data_2_nodup.loc[mask1, 'surface_covered_in_m2'])

data_2_nodup.loc[:, 'price_usd_per_m2_clean'].isnull().sum()/data_2_nodup.shape[0]

### Analicemos outliers de la columna 'price_usd_per_m2_clean'

In [ ]:
describe_price_usd_per_m2_clean = data_2_nodup["price_usd_per_m2_clean"].describe()
describe_price_usd_per_m2_clean

In [ ]:
q1 = describe_price_usd_per_m2_clean.quantile(0.25)
print(q1)
q2 = describe_price_usd_per_m2_clean.quantile(0.5)
print(q2)
q3 = describe_price_usd_per_m2_clean.quantile(0.75)
print(q3)


iqr = (q3 - q1) * 1.5

up_threshold = q3 + iqr
low_threshold = q1 - iqr

print(up_threshold)
print(low_threshold)

outlier_mask_up = describe_price_usd_per_m2_clean > up_threshold
outlier_mask_down = describe_price_usd_per_m2_clean < low_threshold
outlier_mask = np.logical_or(outlier_mask_up, outlier_mask_down)
not_outliers = np.logical_not(outlier_mask)

outliers = describe_price_usd_per_m2_clean[outlier_mask]

In [ ]:
describe_price_usd_per_m2_clean_filtered = describe_price_usd_per_m2_clean[not_outliers]
describe_price_usd_per_m2_clean_filtered.describe()

In [ ]:
describe_price_usd_per_m2_clean.describe()

In [ ]:
sns.boxplot(x=describe_price_usd_per_m2_clean.values);

In [ ]:
sns.boxplot(x=describe_price_usd_per_m2_clean_filtered.values);

In [ ]:
describe_price_usd_per_m2_clean_filtered

# Limpieza con regex

In [ ]:
data_2_nodup['description'][1]

In [ ]:
pileta_pattern = "pileta|picina|pisina|piscina"
pileta_regex = re.compile(pileta_pattern, re.I)
pileta_matches = data_2_nodup['description'].apply(lambda x: x if x is np.NaN else pileta_regex.search(x))
pileta_mask = pileta_matches.notnull()
pileta_description = pileta_matches.loc[pileta_mask]

In [ ]:
pileta_description

In [ ]:
pileta_description.iloc[2].string

In [ ]:
pileta_mask

In [ ]:
description_to_replace = [x.string for x in pileta_description]
print(len(description_to_replace))
description_to_replace_unique = list(set(description_to_replace))
print(len(description_to_replace_unique))

In [ ]:
data_2_nodup['pileta']=''

In [ ]:
data_2_nodup['cochera']=''

In [ ]:
data_2_nodup.loc[pileta_mask, 'pileta'] =1


In [ ]:
data_2_nodup.head(1)

In [ ]:
cochera_pattern = "cochera|garage"
cochera_regex = re.compile(cochera_pattern, re.I)
cochera_matches = data_2_nodup['description'].apply(lambda x: x if x is np.NaN else cochera_regex.search(x))
cochera_mask = cochera_matches.notnull()
cochera_description = cochera_matches.loc[cochera_mask]

In [ ]:
cochera_description.iloc[0].string

In [ ]:
description_to_replace_cochera= [x.string for x in cochera_description]
print(len(description_to_replace_cochera))
description_to_replace_unique_cochera = list(set(description_to_replace_cochera))
print(len(description_to_replace_unique_cochera))

In [ ]:
data_2_nodup.loc[cochera_mask, 'cochera'] = 1

In [ ]:
data_2_nodup.head(3)

### regex ambientes

In [ ]:
pattern_ambientes = "(?P<ambiente>\w+?\s(amb|AMB|Ambiente|ambiente|AMBIENTE|Ambientes|ambientes|AMBIENTES|MONOAMBIENTE)\s\d?)"
regex_ambientes = re.compile(pattern_ambientes)
description = data_2_nodup['description'].astype(str)
resultado_ambientes = description.apply(lambda x: regex_ambientes.search(x))
ambientes_match = resultado_ambientes.apply(lambda x: x if x is None else x.group('ambiente'))
ambientes_match.value_counts

In [ ]:
pattern_ambientes = "(?P<ambiente>\w+?\s(amb|AMB|Ambiente|ambiente|AMBIENTE|Ambientes|ambientes|AMBIENTES|MONOAMBIENTE)\s\d?)"
regex_ambientes = re.compile(pattern_ambientes)
title = data_2_nodup['title'].astype(str)
resultado_ambientes2 = title.apply(lambda x: regex_ambientes.search(x))
ambientes_match2 = resultado_ambientes2.apply(lambda x: x if x is None else x.group('ambiente'))
ambientes_match2.value_counts

In [ ]:
ambientes_match.head(10)

In [ ]:
data_2_nodup['ambientes'] = ambientes_match
data_2_nodup['ambientes2'] = ambientes_match2

In [ ]:
data_ambientes = data_2_nodup.loc[:, 'ambientes': 'ambientes2'].astype(str)
data_ambientes

In [ ]:
pattern_nro_ambientes = "(?P<nro_ambientes>\d)"
regex_nro_ambientes = re.compile(pattern_nro_ambientes)

resultado_nro_amb = data_ambientes['ambientes'].apply(lambda x: regex_nro_ambientes.search(x))
# [('ambientes', 'ambientes2')]
nro_amb_match = resultado_nro_amb.apply(lambda x: x if x is None else x.group('nro_ambientes'))


In [ ]:
pattern_nro_ambientes = "(?P<nro_ambientes>\d)"
regex_nro_ambientes = re.compile(pattern_nro_ambientes)

resultado_nro_amb2 = data_ambientes['ambientes2'].apply(lambda x: regex_nro_ambientes.search(x))

nro_amb_match2 = resultado_nro_amb2.apply(lambda x: x if x is None else x.group('nro_ambientes'))

nro_amb_match2

In [ ]:
df_amb = pd.DataFrame([nro_amb_match, nro_amb_match2])
df_ambientes = df_amb.T
df_ambientes

In [ ]:
df_ambientes.apply(lambda x: x['ambientes'] if x['ambientes']!= None else x['ambientes2'],axis=1)

Nos queda meterlo en el dataset final

# Geo analysis
    

In [ ]:
geo_prop=data_2_nodup[['lon','lat','geonames_id']].copy()
geo_prop.head()

In [ ]:
from shapely.geometry import Point


In [ ]:
geo_prop = gpd.points_from_xy(data_2_nodup.lon, data_2_nodup.lat)
dptos = gpd.GeoDataFrame(data_2_nodup, geometry=geo_prop)

In [ ]:
data_2_nodup=data_2_nodup.drop(index=[28512])
data_2_nodup=data_2_nodup.drop(index=[9761])

In [ ]:
dptos.head(2)

In [ ]:
import contextily  as cx

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
ax = world[world.name == 'Argentina'].plot(color='White', edgecolor='black')
dptos.plot(ax=ax, color='green')
plt.show()

In [ ]:
# Muestra los pozos en un gráfico con latitud y longitud
fig, ax = plt.subplots(figsize=(4,6))
ax.set_title('Propiedades')
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')
plt.grid()

dptos.plot(ax=ax,
               legend = True,
               alpha=0.3)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,15))
ax.set_title('Propiedades')
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')
plt.grid()

dptos.crs = "EPSG:4326"                        
dptos_3857 = dptos.to_crs(epsg=3857)       
dptos_3857.plot(column='price_usd_per_m2_clean',ax=ax,legend = True,alpha=0.3)

cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite) 

In [ ]:
operation_type = data_2_nodup.groupby('property_type', axis=0).count().iloc[:, 0]
operation_type

In [ ]:
data_2_nodup.loc[data_2_nodup['place_name'].isnull(), 'place_name'] = 'Tigre'

In [ ]:
data2_nodup.groupby(['place_name', 'property_type'])['price_usd_per_m2'].mean().round(2)

In [ ]:
mask_place1= data_2_nodup['place_name'].value_counts() ==1
mask_place1.value_counts()

mask1= data_2_nodup['place_name']  == ' Country Maschwitz Club'

data_2_nodup[mask1]

In [ ]:
data_2_nodup.place_with_parent_names.str.split(expand= True )

In [ ]:
df_data = pd.DataFrame(data_2_nodup.place_with_parent_names.str.split('|',4).tolist(),
                         columns = ['first','2','3','place_final', 'Last'])

labels_drop = ['2', '3', 'Last']
df_data1 = df_data.drop(labels= labels_drop, axis=1)

In [ ]:
result = pd.concat([data_2_nodup, df_data1], axis=1)
result

In [ ]:
result['place_final'].value_counts()

In [ ]:
# PROBAMOS CON GROUP

# data_group = result.groupby(['place_final', 'property_type'])['price_usd_per_m2'].mean().round(2)

In [ ]:
data_pivot = result.pivot_table(index = ['place_final', 'property_type' ], aggfunc={'price_usd_per_m2':'mean'}).round(2)
data_pivot

In [ ]:
data_pivot.loc['Abasto']